In [1]:
%config Completer.use_jedi = False

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os

import torch

import sparse

import numpy as np

import tqdm.autonotebook as tqdm

import matplotlib.pyplot as plt

import torchio
import torchio.transforms

from utils import segmentate_patient, resample, CTDataset, plot_3d, subplots_3d

If you use TorchIO for your research, please cite the following paper:
Pérez-García et al., TorchIO: a Python library for efficient loading,
preprocessing, augmentation and patch-based sampling of medical images
in deep learning. Credits instructions: https://torchio.readthedocs.io/#credits



In [4]:
test_patients = sorted(os.listdir('./data/test/'))
train_patients = sorted(os.listdir('./data/train/'))

In [5]:
# mode = 'train'
# PROCESSED_PATH = './processed_data/'

# for patient_n in tqdm.tqdm(range(len(train_patients) if mode == 'train' else len(test_patients))):
#     patient = train_patients[patient_n] if mode == 'train' else test_patients[patient_n]
    
#     all_images, _, _, all_masks, meta_data = segmentate_patient(mode, patient_n)
#     SliceThickness, PixelSpacing = meta_data['SliceThickness'][0], meta_data['PixelSpacing'][0]
#     assert len(PixelSpacing) == 2
    
#     new_spacing = np.array([SliceThickness] + list(PixelSpacing))
#     # noinspection PyBroadException
#     try:
#         ordering, case = np.argsort([float(_) for _ in meta_data['SliceLocation']]), 0
#     except Exception:
#         # noinspection PyBroadException
#         try:
#             ordering, case = np.argsort([float(_) for _ in meta_data['InstanceNumber']]), 1
#         except Exception:
#             ordering, case = np.arange(len(all_images)), 2
    
#     all_images, all_masks = np.array(all_images)[ordering], np.array(all_masks)[ordering]
#     for key, values in meta_data.items():
#         meta_data[key] = np.array(values)[ordering].tolist()
    
#     if len(all_images) != 196:
#         all_images, _ = resample(
#             all_images, [196, *all_images.shape[1:]], SliceThickness, PixelSpacing
#         )
#         all_masks, new_spacing = resample(
#             all_masks, [196, *all_images.shape[1:]], SliceThickness, PixelSpacing
#         )
#         all_masks = all_masks > 0

#         meta_data['SliceThickness'] = [new_spacing[0] for _ in meta_data['SliceThickness']]
#         meta_data['PixelSpacing'] = [[new_spacing[0], new_spacing[1]] for _ in meta_data['PixelSpacing']]

#     base_path = os.path.join(PROCESSED_PATH, mode, train_patients[patient_n])
#     os.makedirs(base_path, exist_ok=True)
    
#     if all_images.shape[1] == 512:
#         all_masks = all_masks[:, ::2, ::2]
#         all_images = all_images[:, ::2, ::2]
#         meta_data['PixelSpacing'] = [[new_spacing[0] * 2, new_spacing[1] * 2] for _ in meta_data['PixelSpacing']] 
#     if all_images.shape[1] == 632:
#         all_masks = all_masks[:, 60:-60:2, 60:-60:2]
#         all_images = all_images[:, 60:-60:2, 60:-60:2]
#         meta_data['PixelSpacing'] = [[new_spacing[0] * 2, new_spacing[1] * 2] for _ in meta_data['PixelSpacing']]
#     if all_images.shape[1] == 768:
#         all_masks = all_masks[:, ::3, ::3]
#         all_images = all_images[:, ::3, ::3]
#         meta_data['PixelSpacing'] = [[new_spacing[0] * 3, new_spacing[1] * 3] for _ in meta_data['PixelSpacing']]
#     if all_images.shape[1] == 1302:
#         all_masks = all_masks[:, 11:-11:5, 11:-11:5]
#         all_images = all_images[:, 11:-11:5, 11:-11:5]
#         meta_data['PixelSpacing'] = [[new_spacing[0] * 5, new_spacing[1] * 5] for _ in meta_data['PixelSpacing']]

#     np.save(os.path.join(base_path, 'meta.npy'), meta_data)
#     np.save(os.path.join(base_path, 'images.npy'), all_images)
#     sparse.save_npz(os.path.join(base_path, 'masks.npz'), sparse.COO(np.array(all_masks)))

In [6]:
transforms = torchio.transforms.Compose([
    torchio.transforms.RandomAffine(
        degrees=(10, 10),
        translation=(-10, -10),
        isotropic=False,
        default_pad_value='minimum',
        image_interpolation='linear',
    ),
])

In [7]:
# dataset_test = CTDataset(
#     './processed_data/train', 
#     './processed_data/train.csv', 
#     train=False, test_size=0.25, random_state=42
# )
dataset_train = CTDataset(
    './processed_data/train', 
    './processed_data/train.csv', 
    train=True, transform=transforms,
    test_size=0, random_state=42,
    padding_mode=None, padding_constant=None
)

In [8]:
obj = dataset_train[0]
obj_transf = dataset_train[0]

In [10]:
from torch.utils.data import DataLoader

dl = DataLoader(dataset_train, batch_size=1, num_workers=3)

In [ ]:
for obj in tqdm.tqdm(dl):
    print(obj[0])

In [ ]:
fig, axes = subplots_3d(2, 2, figsize=(20, 14))
plot_3d(axes[0][0], obj.masks.numpy()[0], 4, threshold=0.0)
plot_3d(axes[0][1], obj_transf.masks.numpy()[0], 4, threshold=0.0)

plot_3d(axes[1][0], obj.images.numpy()[0], 4, threshold=-700.0)
plot_3d(axes[1][1], obj_transf.images.numpy()[0], 4, threshold=-700.0)

plt.show()

tensor([[-4,  5,  7,  9, 11, 17, 29, 41, 57]])
tensor([[ 8,  9, 11, 13, 15, 22, 33, 45, 60]])
tensor([[ 0,  1,  3,  5,  7, 13, 25, 37, 54]])
tensor([[ 6,  7,  9, 11, 13, 19, 32, 43, 58]])
tensor([[33, 35, 37, 39, 41, 47, 58, 71, 87]])
tensor([[ 0,  2,  4,  6,  8, 14, 26, 37, 54]])
tensor([[12, 17, 19, 21, 31, 40, 52, 69]])
tensor([[13, 14, 16, 18, 20, 26, 38, 53, 66]])
tensor([[18, 19, 21, 23, 25, 31, 44, 54, 70]])
tensor([[-3,  3,  5,  7,  9, 15, 27, 39, 55]])
tensor([[ 0,  1,  3,  6,  7, 14, 26, 38, 53]])
tensor([[41, 45, 47, 49, 51, 57, 69, 81, 98]])
tensor([[12, 14, 16, 18, 23, 26, 38, 51, 66]])
tensor([[19, 30, 33, 34, 36, 42, 54, 65]])
tensor([[30, 35, 37, 39, 41, 47, 59, 71, 87]])
tensor([[11, 12, 14, 16, 18, 24, 36, 49, 65]])
tensor([[ 6, 12, 15, 17, 19, 24, 36, 49, 63]])
tensor([[ 1,  6,  9, 11, 12, 18, 31, 44, 60]])
tensor([[ 2,  4,  6,  8, 10, 29]])
tensor([[ 9,  9, 11, 13, 15, 21, 33, 47, 61]])
tensor([[-1,  1,  3,  6,  7, 13, 26, 37, 54]])
tensor([[48, 51, 53, 56, 58, 63, 

In [ ]:
fig, axes = subplots_3d(2, 2, figsize=(20, 14))
plot_3d(axes[0][0], obj.masks.numpy()[0], 4, threshold=0.0)
plot_3d(axes[0][1], obj_transf.masks.numpy()[0], 4, threshold=0.0)

plot_3d(axes[1][0], obj.images.numpy()[0], 4, threshold=-700.0)
plot_3d(axes[1][1], obj_transf.images.numpy()[0], 4, threshold=-700.0)

plt.show()